## 토픽 모델링(Topic Modeling)
토픽(Topic)은 한국어로는 주제라고 합니다. 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다.

사이킷런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 제공합니다. 앞서 언급했듯이 LSA가 토픽 모델링에 최적화 된 알고리즘은 아니지만, 토픽 모델링이라는 분야의 시초가 되는 알고리즘입니다. 여기서는 LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링을 수행해보도록 하겠습니다.

In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

훈련에 사용할 뉴스는 총 11,314개입니다. 이 중 첫번째 훈련용 뉴스를 출력해보겠습니다.

In [15]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [71]:
len(documents[1])

546

뉴스 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어져 있습니다. 이런 형식의 뉴스가 총 11,314개 존재합니다. 사이킷런이 제공하는 뉴스 데이터에서 target_name에는 본래 이 뉴스 데이터가 어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.

In [5]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


## 2) 텍스트 전처리
시작하기 앞서, 텍스트 데이터에 대해서 가능한한 정제 과정을 거쳐야만 합니다. 기본적인 아이디어는 알파벳을 제외한 구두점, 숫자, 특수 문자를 제거하는 것입니다. 이는 텍스트 전처리 챕터에서 정제 기법으로 배웠던 정규 표현식을 통해서 해결할 수 있습니다. 또한 짧은 단어는 유용한 정보를 담고있지 않다고 가정하고, 길이가 짧은 단어도 제거합니다. 그리고 마지막으로 모든 알파벳을 소문자로 바꿔서 단어의 개수를 줄이는 작업을 합니다.

In [0]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

데이터의 정제가 끝났습니다. 다시 첫번째 훈련용 뉴스만 출력하여 정제 전, 후에 어떤 차이가 있는지 확인해보겠습니다.

In [19]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [67]:
len(news_df['clean_doc'][1])

316

우선 특수문자가 제거되었으며, if나 you와 같은 길이가 3이하인 단어가 제거된 것을 확인할 수 있습니다. 뿐만 아니라 대문자가 전부 소문자로 바뀌었습니다. 이제 뉴스 데이터에서 불용어를 제거합니다. 불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 토큰화와 불용어 제거를 순차적으로 진행합니다.

In [0]:
from nltk.corpus import stopwords

In [27]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
print(stopwords)

<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>


In [74]:
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
print(tokenized_doc[1])
print(len(tokenized_doc[1]))

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'your', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'that', 'have', 'these', 'feelings', 'denial', 'about', 'faith', 'need', 'well', 'just', 'pretend', 'that', 'will', 'happily', 'ever', 'after', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'your', 'flintstone', 'chewables', 'bake', 'timmons']
50


In [41]:
# print(stop_words)
# print(len(stop_word))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [0]:
# tokenized_doc[1]

In [69]:
len(tokenized_doc[1])

40

기존에 있었던 불용어에 속하던 your, about, just, that, will, after 단어들이 사라졌을 뿐만 아니라, 토큰화가 수행된 것을 확인할 수 있습니다.

### 내포 형식과 apply lambda정리

In [44]:
a = [1,2,3,4]
result=[num*3 for num in a if num%2==0]
result

[6, 12]

In [63]:
df3 = pd.DataFrame({
    'A': [1, 3, 4, 3, 44],
    'B': [2, 3, 1, 22, 3],
    'C': [1, 56, 2, 4, 4]
})
df3

,A,B,C
0,1,2,1
1,3,3,56
2,4,1,2
3,3,22,4
4,44,3,4


In [65]:
df3.apply(lambda x: x+1000)

,A,B,C
0,1001,1002,1001
1,1003,1003,1056
2,1004,1001,1002
3,1003,1022,1004
4,1044,1003,1004


## 3) TF-IDF 행렬 만들기
불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 챕터 참고)는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서 다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다

In [77]:
news_df

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...",danny rubenstein israeli journalist will speak...
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree home runs clemens always memorable kinda...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


In [78]:
tokenized_doc

0        [well, sure, about, story, seem, biased, what,...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, that, principle, your, str...
3        [notwithstanding, legitimate, fuss, about, thi...
4        [well, will, have, change, scoring, playoff, p...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, will,...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, with, orange, micros, grappler...
11313    [argument, with, murphy, scared, hell, when, c...
Name: clean_doc, Length: 11314, dtype: object

In [0]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [81]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

상적으로 불용어가 제거된 상태에서 역토큰화가 수행되었음을 확인할 수 있습니다.

이제 사이킷런의 TfidfVectorizer를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들 것입니다. 물론 텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수는 있겠지만, 여기서는 1,000개의 단어로 제한하도록 하겠습니다.

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

11,314 × 1,000의 크기를 가진 TF-IDF 행렬이 생성되었음을 확인할 수 있습니다.

## 4) 토픽 모델링(Topic Modeling)
이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 여기서는 사이킷런의 절단된 SVD(Truncated SVD)를 사용합니다. 절단된 SVD를 사용하면 차원을 축소할 수 있습니다. 원래 기존 뉴스 데이터가 20개의 뉴스 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도해보겠습니다. 토픽의 숫자는 n_components의 파라미터로 지정이 가능합니다.

In [85]:
from sklearn.decomposition import TruncatedSVD
import numpy as np
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [86]:
#여기서 svd_model.componets_는 앞서 배운 LSA에서 VT에 해당됩니다.

np.shape(svd_model.components_)

(20, 1000)

In [87]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('just', 0.20887), ('like', 0.20469), ('know', 0.19349), ('people', 0.18318), ('think', 0.1697)]
Topic 2: [('thanks', 0.32763), ('windows', 0.28786), ('card', 0.18019), ('drive', 0.16864), ('mail', 0.15261)]
Topic 3: [('game', 0.34011), ('team', 0.30311), ('year', 0.26894), ('games', 0.23784), ('drive', 0.17472)]
Topic 4: [('drive', 0.46159), ('scsi', 0.17188), ('disk', 0.14451), ('hard', 0.13805), ('problem', 0.12763)]
Topic 5: [('drive', 0.39993), ('know', 0.28768), ('thanks', 0.24917), ('does', 0.24678), ('just', 0.17387)]
Topic 6: [('just', 0.55559), ('like', 0.23559), ('windows', 0.23078), ('know', 0.15795), ('does', 0.11156)]
Topic 7: [('just', 0.43264), ('like', 0.22858), ('mail', 0.15052), ('bike', 0.11698), ('thanks', 0.10025)]
Topic 8: [('does', 0.39692), ('know', 0.25192), ('chip', 0.22492), ('like', 0.17824), ('card', 0.15695)]
Topic 9: [('like', 0.42065), ('card', 0.32249), ('sale', 0.20267), ('video', 0.1571), ('offer', 0.14119)]
Topic 10: [('like', 0.61166), ('

각 20개의 행의 각 1,000개의 열 중 가장 값이 큰 5개의 값을 찾아서 단어로 출력합니다.

정리해보면 LSA는 쉽고 빠르게 구현이 가능할 뿐만 아니라 단어의 잠재적인 의미를 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있습니다. 하지만 SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고하면 보통 처음부터 다시 계산해야 합니다. 즉, 새로운 정보에 대해 업데이트가 어렵습니다. 이는 최근 LSA 대신 Word2Vec 등 단어의 의미를 벡터화할 수 있는 또 다른 방법론인 인공 신경망 기반의 방법론이 각광받는 이유이기도 합니다.